In [26]:
    import IPython, toolz.curried as toolz, base64, vdom
    try: from .weave import markdown_to_flex_vdom
    except: from weave import markdown_to_flex_vdom

In [27]:
    mimes = toolz.pipe("""
        50 text/html
        70 text/latex
        80 image/svg+xml
        90 image/bmp image/png image/jpeg image/gif'
        60 text/markdown
        110 text/javascript application/javascript 
        120 text/plain
        """, str.strip, str.splitlines, toolz.map(toolz.compose(toolz.juxt(toolz.compose(int, next), list), iter, str.split)), list)

In [28]:
    image_types = list(map('image/'.__add__, ('bmp', 'jpeg', 'gif', 'png')))

In [29]:
    def flatten(str): return ''.join(str.splitlines())

In [30]:
    import vdom, base64, IPython

    def flatten(string):
        return ''.join(map(str.strip, string.splitlines()))

    def format_images(type, bundle):
        str = bundle[type]
        if isinstance(str, bytes):
            str = base64.b64encode(str).decode('utf-8')
            
        if type in ('image/svg+xml', 'text/html'): 
            return IPython.display.HTML(flatten(str))
            
        if str.startswith('http'): return vdom.img(src=str)

        return vdom.img(src=f"data:{type};base64,{str}")

In [31]:
  def ipython_formatter(object):
        import mistune
        format = IPython.get_ipython().display_formatter.format
        bundle, metadata = format(object)
        for type in toolz.pipe(mimes, toolz.map(toolz.second), toolz.concat):
            if type == 'text/plain':  return object
            if type in bundle: 
                object = bundle[type]
                if type.startswith('image'):
                    return format_images(type, bundle)._repr_html_()
                if type == 'text/markdown':
                    return markdown_to_flex_vdom()(object)._repr_html_()
                return bundle[type]